/usr/local/bin/python3 -m pip install csv

In [ ]:
import csv
import unicodedata
import requests
import xml.etree.ElementTree

ourairports_base_url = 'https://github.com/davidmegginson/ourairports-data/raw/main/'
ourairports_datafiles = ['airports.csv', 'countries.csv', 'regions.csv']
wikipedia_airline_codes_url = 'https://en.wikipedia.org/wiki/List_of_airline_codes'

image_path = '../lib/airline-logos/images/'
   
force_download = False
download_logos = False

### Download files from github.com (ourairports) and WikiPedia

In [ ]:
if force_download:
    for f in ourairports_datafiles:
        url = ourairports_base_url + f
        r = requests.get(url)
        if (r.status_code == 200):
            with open('.' + f, 'w') as fp:
                fp.write(r.text)

    r = requests.get(wikipedia_airline_codes_url)
    if (r.status_code == 200):
        with open('.airlines.html', 'w') as fp:
            fp.write(r.text)

In [ ]:
if download_logos:
    for airline in airlines:
        logo_path = image_path + airline['icao'] + '.png'
        # Do not overwrite
        if os.path.exists(logo_path):
            continue

        logo_url = 'https://e0.flightcdn.com/images/airline_logos/90p/'+ airline['icao'] + '.png'
        #logo_url = 'https://flightaware.com/live/fleet/' + key + '.png'
        request = requests.get(logo_url, allow_redirects=True)
        if request.status_code == 200:
            open(logo_path, 'wb').write(request.content)
             print ('Downloaded ' + logo_url)

## Process files

In [ ]:
countries = {}

# "id","code","name","continent","wikipedia_link","keywords"
with open('.countries.csv', 'r', encoding='utf-8') as csvfile:
    countries_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in countries_reader:
        countries[row['code']] = row

countries = sorted(countries.values(), key=lambda x: x['code'])

In [ ]:
airports = {}

# "id","ident","type","name","latitude_deg","longitude_deg","elevation_ft","continent","iso_country","iso_region","municipality","scheduled_service","gps_code","iata_code","local_code","home_link","wikipedia_link","keywords"
with open('.airports.csv', 'r', encoding='utf-8') as csvfile:
    airports_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in airports_reader:
        if len(row['iata_code']) == 3:
            airports[row['iata_code']] = row

airports = sorted(airports.values(), key=lambda x: x['iata_code'])

Prepare map for resolving countries to iso codes

In [ ]:
countrymap = {}
for c in countries:
    countrymap[c['name']] = c['code']

# Incorrect mappings or typos
countrymap['Russian Federation'] = 'RU'
countrymap['Ivory Coast'] = 'CI'
countrymap['Global'] = 'ZZ'
countrymap['Republic of Korea'] = 'KR'
countrymap['Macao'] = 'AS'
countrymap['Netherlands Antilles'] = 'NA'
countrymap['São Tomé and Príncipe'] = 'ST'
countrymap['Democratic Republic of the Congo'] = 'CD'
countrymap['Reunion'] = 'RE'
countrymap['Somali Republic'] = 'SO'
countrymap['The Gambia'] = 'GM'
countrymap['Peru'] = 'PE'
countrymap['Swaziland'] = 'SZ'
countrymap['Macedonia'] = 'MK'
countrymap['Myanmar']= 'MM'
countrymap['Democratic People\'s Republic of Korea'] = 'KP'
countrymap['Moldovoa']= 'MD'
countrymap['Burkino Faso'] = 'BF'
countrymap['Syrian Arab Republic'] = 'SY'
countrymap['CANADA'] = 'CA'
countrymap['Republic of the Congo'] = 'CG'
countrymap['Democratic Republic of Congo'] = 'CD'
countrymap['UAE'] = 'AE'
countrymap['Hong Kong SAR of China']= 'HK'
countrymap['French Guyana'] = 'GF'
countrymap['Lao People\'s Democratic Republic'] = 'LA'
countrymap['THAILAND'] = 'TH'
countrymap['Molodva'] = 'MD'
countrymap['the Netherlands'] = 'NL'
countrymap['Domocratic Republic of Congo'] = 'CD'
countrymap['Spain and France'] = 'ES'   # Level airline
countrymap['Sweden, Denmark and Norway'] = 'SE' #SAS
countrymap['Norway, Sweden and San Diego'] = 'GB' # SCAVAC flight school

In [ ]:

root = xml.etree.ElementTree.parse('.airlines.html').getroot()

airlines=[]

# IATA	ICAO	Airline	Call sign	Country/Region	Comments
rows = root.findall('.//table//tr')
rows.pop(0)
fields = ['iata_code', 'icao_code', 'name', 'callsign', 'country', 'comments']
for row in rows:
    cells = row.findall('.//td')
    data = {}
    for i in range(0, len(fields)- 1):
        cell = cells[i]
        while True:
            val  = cell.text
            if val is not None:
                break
            # sometimes in a <a> tag, and sometimes in a <a><s></s></a>
            cell = cell.find('*')
            if cell is None:
                val = ''
                break;
            
        data[fields[i]] = val.strip()

    if data['icao_code']:
        # Exceptions in HTML
        if data['icao_code'] == '—' or data['icao_code'] == 'n/a' or data['country'] == 'n/a':
            continue

        if not data['country'] in countrymap.keys():
            print('Country not found: ' + data['country'])
            continue

        data['iso_country'] = countrymap[data['country']]
        airlines.append(data)

airlines = sorted(airlines, key=lambda x: x['icao_code'])
#print(airlines)

## Write data

In [ ]:
for country in countries:
      line = 'const country_t country_' + country['code'] + ' = { "' + country['code'] + '", "' + country['name']+ '", &country_flag_' + country['code'] + ' };'
      print(unicodedata.normalize('NFKD', line).encode('ascii', errors='ignore').decode('ascii'))


In [ ]:
for airport in airports:
    region = airport['iso_region'].split('-')[1]
    line = '{ "' + airport['iata_code'] + '", "' + airport['name'].replace('"', '\'') + '", "' + airport['municipality']+ '", "' + region + '", &country_' + airport['iso_country'] + ' ,' +  str(airport['latitude_deg']) + ', ' + str(airport['longitude_deg']) + ', ' + str(airport['elevation_ft'] + ' },')
    print(unicodedata.normalize('NFKD', line).encode('ascii', errors='ignore').decode('ascii'))

In [ ]:
for airline in airlines:
    logo_path = image_path + airline['icao_code'] + '.png'
    logo = '&airline_logo_' + airline['icao_code'] if os.path.exists(logo_path) else 'nullptr'

    values = ['"' + airline['icao_code'] + '"', '"' + airline['callsign'] + '"', '"' + airline['name'].replace('"', '\'') + '"', '&country_' + airline['iso_country'],  logo]
    line = "{ " + ", ".join(values) + "},"
    print(unicodedata.normalize('NFKD', line).encode('ascii', errors='ignore').decode('ascii'))
